# Task 2: Recommendation Engine - Skeleton Notebook

This notebook provides a very basic example for the notebook you are expected to submit for Task 2 of the Final Project. The main purpose is that we can try different examples to get a better sense of your approach. Compared to Task 1 (Kaggle Competition), we don't have any objective means to evaluate the recommendations. 

Some general comments:
* You can import any data you need. This particularly includes your cleaned version of the properties dataset (incl. the auxiliary data or any other data you might have collected); there's no need to show the data cleaning / preprocessing steps in this notebook.
* You can also import your code in form of external Python (.py) script. You're actually encouraged to do so to keep this notebook light and uncluttered.
* **Important:** Please consider this notebook as an example and not to set specific requirements. Your notebook is likely to look very different. As long there is a section where we can easily test your solution, it should be fine.

## Setting up the Notebook

In [ ]:
import numpy as np
import pandas as pd
#from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist

## Load the Data

For this example, we use a simplified version of the dataset with only 2k+ data samples, each with only a subset of features.

In [ ]:
#df_sample = pd.read_csv('data/sg-property-prices-simplified.csv')

#df_sample.head()

df_pristine = pd.read_csv('data/train.csv') # We display this.
df_numerical = pd.read_csv('data/processed.csv') # We work on this.

## Computing the Top Recommendations

The method `get_top_recommendations()` shows an example of how to get the top recommendations for a given data sample (data sample = row in the dataframe of the dataset). The input is a row from the dataset and a list of optional input parameters which will depend on your approach; `k` is the number of returned recommendations seems useful, though.

The output should be a `pd.DataFrame` containing the recommendations. The output dataframe should have the same columns as the row + any additional columns you deem important (e.g., any score or tags that you might want to add to your recommendations).

In principle, the method `get_top_recommendations()` may be imported from a external Python (.py) script as well.

In [ ]:
def get_top_recommendations(row, **kwargs) -> pd.DataFrame:
    
    #####################################################
    ## Initialize the required parameters
    
    # The number of recommendations seem recommended
    # Additional input parameters are up to you
    k = None
    cold_start = True
    
    # Extract all **kwargs input parameters
    # and set the used paramaters (here: k)
    for key, value in kwargs.items():
        if key == 'k':
            k = value
        elif key == 'cold_start':
            cold_start = value
            
    #####################################################
    ## Compute your recommendations
    #
    # This is where your magic happens. Of course, you can call methods
    # defined in this notebook or in external Python (.py) scripts
    
    idx = df_pristine.loc[df_pristine['listing_id'] == row['listing_id']].index[0]

    # No past viewing history available. Recommend based on current listing.
    if cold_start:
        # Geographical Slice
        geo_slice_cols = ('property_type', 'num_beds', 'num_baths', '')
        df_geo = df_numerical.loc[:,geo_slice_cols].to_numpy().reshape(-1,1)
        sim_geo = pd.DataFrame(1-cdist(df_geo, df_geo, metric='cosine'))
        sim_geo_topk = pd.DataFrame(sim_geo.columns.to_numpy()[np.argsort(sim_geo[idx].to_numpy())][:, -1:-k-1:-1])
        
    df_result = []
    for i in range(k):
        rec = sim_geo_topk[i][idx]
        df_result.append(df_pristine.iloc[rec])
    df_result = pd.concat(df_result)
    df_result = pd.DataFrame(df_result.values.reshape(k, 21))
    
    # Return the dataset with the k recommendations
    return df_result


## Testing the Recommendation Engine

This will be the main part of your notebook to allow for testing your solutions. Most basically, for a given listing (defined by the row id in your input dataframe), we would like to see the recommendations you make. So however you set up your notebook, it should have at least a comparable section that will allow us to run your solution for different inputs.

### Pick a Sample Listing as Input

In [ ]:
# Pick a row id of choice
#row_id = 10
row_id = 0
#row_id = 30
#row_id = 40
#row_id = 50

# Get the row from the dataframe (an valid row ids will throw an error)
row = df_pristine.iloc[row_id]

# Just for printing it nicely, we create a new dataframe from this single row
pd.DataFrame([row])

## Compute and Display the recommendations

Since the method `get_top_recommendations()` returns a `pd.DataFrame`, it's easy to display the result.

In [ ]:
k = 10

df_recommendations = get_top_recommendations(row, k=k)

df_recommendations.head(k)